# Qiskit Workshop: Solving Optimization Problems

## Quadratic Unconstrained Binary Optimization (QUBO)

**Problem:** Given some combinatorial optimization problem with cost function $C(x)$,

<center>
    $\text{maximize } \;\;      C(x)$
 </center>
 <center>
    $\text{subject to } \;\; x \in S$
</center>

If $x$ are binary variables (i.e. $x \in \{0,1\}^n$), problem can be expressed as a Quadratic Unconstrained Binary Optimization (QUBO) problem.

In [ ]:
from qiskit_optimization import QuadraticProgram

qubo = QuadraticProgram()
qubo.binary_var('x')
qubo.binary_var('y')
qubo.binary_var('z')
qubo.minimize(linear=[1,-2,3], quadratic={('x', 'y'): 1, ('x', 'z'): -1, ('y', 'z'): 2})
print(qubo.export_as_lp_string())

In [ ]:
op, offset = qubo.to_ising()
print('offset: {}'.format(offset))
print('operator:')
print(op)

In [ ]:
from qiskit.utils import algorithm_globals
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms.minimum_eigensolvers import SamplingVQE
from qiskit.primitives import Sampler

algorithm_globals.random_seed = 10598

# Create classical optimizer for VQE
spsa = SPSA(maxiter=300)

# Generate ansatz circuit
ansatz = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz', reps=5, entanglement='linear')

# Instantiate VQE
vqe = SamplingVQE(sampler=Sampler(), ansatz=ansatz, optimizer=spsa)

In [ ]:
# If you want to see the circuit, set the number of qubits and draw

ansatz.num_qubits = op.num_qubits
ansatz.decompose().draw()

In [ ]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer

vqe_meo = MinimumEigenOptimizer(vqe)

In [ ]:
result = vqe_meo.solve(qubo)
print(result)

---

## Max-Cut problem

**Problem:** Consider an undirected weighted graph. Partition the graph into two subsets such that the sum of the weights in the edges connecting nodes between the two different subsets is maximized.

In [ ]:
import networkx as nx

import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline

import numpy as np

In [ ]:
# Number of nodes in graph
n = 4

# Generate a graph
G = nx.Graph()
G.add_nodes_from(np.arange(0,n,1))

# Apply edge weights
# Note: tuple is (i,j,weight) where (i,j) is the edge
elist = [(0,1,1.0), (0,2,1.0), (0,3,1.0), (1,2,1.0), (2,3,1.0)]
G.add_weighted_edges_from(elist)

colors = ['r' for node in G.nodes()]
pos = nx.spring_layout(G)

def draw_graph(G, colors, pos):
    default_axes = plt.axes()
    nx.draw_networkx(G, node_color=colors, node_size=600, ax=default_axes, pos=pos)
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edge_labels)

draw_graph(G, colors, pos)

### Step 0: Generate the weight matrix of the graph

In [ ]:
w = np.zeros([n,n])

for i in range(n):
    for j in range(n):
        temp = G.get_edge_data(i,j,default=0)
        if temp != 0:
            w[i,j] = temp['weight']

print(w)

### Step 1: Use the `Maxcut` to generate a Quadratic Program

In [ ]:
! pip install qiskit_optimization

In [ ]:
from qiskit_optimization.applications import Maxcut
from qiskit_optimization.problems import QuadraticProgram

# Intantiate Maxcut
#max_cut = Maxcut(graph=w)

# Can also provide the NetworkX graph object instead
max_cut = Maxcut(graph=G)

# Convert to quadratic program
qp = max_cut.to_quadratic_program()

print(qp.export_as_lp_string())

### Step 2: Create solver

Construct a minimum eigensolver instance to solve the problem.
In this case, use `VQE`.

In [ ]:
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms.minimum_eigensolvers import SamplingVQE

# Create classical optimizer for VQE
spsa = SPSA(maxiter=300)

# Generate ansatz circuit
ansatz = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz', reps=2, entanglement='linear')

# Instantiate VQE
vqe = SamplingVQE(sampler=Sampler(), ansatz=ansatz, optimizer=spsa)

### Step 3: Compute optimal solution

In [ ]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer

# create minimum eigen optimizer based on VQE
vqe_optimizer = MinimumEigenOptimizer(vqe)

In [ ]:
# solve quadratic program
result = vqe_optimizer.solve(problem=qp)
print(result)

# Plot graph of solution
colors = ['r' if result.x[i] == 0 else 'c' for i in range(n)]
draw_graph(G, colors, pos)

### Solving with QAOA

In [ ]:
from qiskit.algorithms.minimum_eigensolvers import QAOA

qaoa = QAOA(sampler=Sampler(), optimizer=spsa)
qaoa_meo = MinimumEigenOptimizer(qaoa)

qaoa_result = qaoa_meo.solve(qp)
print(qaoa_result)

In [ ]:
qaoa.ansatz.draw()

### Using IBM Runtime for cloud simulator and real hardware

In [ ]:
from qiskit_ibm_runtime import Sampler as SamplerRT
from qiskit_ibm_runtime import Session

In [ ]:
def qaoa_callback(count, params, mean, metadata):
    if count % 10 == 0:
        print('Iteration:', count, ', value:', mean)

In [ ]:
with Session(service=service, backend="ibmq_qasm_simulator") as session:
    sampler_rt = SamplerRT(session=session)
    qaoa = QAOA(sampler=sampler_rt, optimizer=spsa, callback=qaoa_callback)
    qaoa_meo = MinimumEigenOptimizer(qaoa)
    qaoa_result = qaoa_meo.solve(qp)

print(qaoa_result)

---

## Traveling Salesman Problem (TSP)

**Problem:** Consider a weighted graph where the edge weights denote the cost of moving along that edge. Compute the lowest-cost round-trip path that minimizes the total cost.

In [ ]:
from qiskit_optimization.applications import Tsp

In [ ]:
# Generating a random graph
n = 4
tsp = Tsp.create_random_instance(n, seed=123)
adj_matrix = nx.to_numpy_array(tsp.graph)
print('distance\n', adj_matrix)

# Plot the graph
colors = ['r' for node in tsp.graph.nodes]
pos = [tsp.graph.nodes[node]['pos']  for node in tsp.graph.nodes]
draw_graph(tsp.graph, colors, pos)

### Step 1: Convert TSP to quadratic program

In [ ]:
qp = tsp.to_quadratic_program()
print(qp.prettyprint())

### Step 2: Construct solver

In [ ]:
from qiskit.algorithms.minimum_eigensolvers import QAOA
from qiskit.algorithms.optimizers import COBYLA

qaoa = QAOA(sampler=Sampler(), optimizer=COBYLA(maxiter=100), callback=qaoa_callback)
qaoa_meo = MinimumEigenOptimizer(qaoa)

### Step 3: Compute solution

In [ ]:
qaoa_result = qaoa_meo.solve(qp)
print(qaoa_result)

In [ ]:
z = tsp.interpret(qaoa_result.x)
print('solution:', z)
print('solution objective:', tsp.tsp_value(z, adj_matrix))

In [ ]:
def draw_tsp_solution(G, order, colors, pos):
    G2 = nx.DiGraph()
    G2.add_nodes_from(G)
    n = len(order)
    for i in range(n):
        j = (i + 1) % n
        G2.add_edge(order[i], order[j], weight=G[order[i]][order[j]]['weight'])
    default_axes = plt.axes(frameon=True)
    nx.draw_networkx(G2, node_color=colors, edge_color='b', node_size=600, alpha=.8, ax=default_axes, pos=pos)
    edge_labels = nx.get_edge_attributes(G2, 'weight')
    nx.draw_networkx_edge_labels(G2, pos, font_color='b', edge_labels=edge_labels)

In [ ]:
draw_tsp_solution(tsp.graph, z, colors, pos)